<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Nodes" data-toc-modified-id="Nodes-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Nodes</a></span></li><li><span><a href="#Data-model" data-toc-modified-id="Data-model-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data model</a></span></li></ul></li><li><span><a href="#Setting-up" data-toc-modified-id="Setting-up-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setting up</a></span><ul class="toc-item"><li><span><a href="#Required-libraries" data-toc-modified-id="Required-libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Required libraries</a></span></li><li><span><a href="#Setting-up-logging" data-toc-modified-id="Setting-up-logging-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setting up logging</a></span></li></ul></li><li><span><a href="#Selecting-and-plotting-data-using-code" data-toc-modified-id="Selecting-and-plotting-data-using-code-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Selecting and plotting data using code</a></span></li></ul></div>

# Introduction

This is a basic illustration of how plottr can be used for interactive data inspection. In this notebook we go through a simple example of how plottr easily allows us to look at multidimensional data.


Let's start with some key concepts that plottr uses under the hood:


## Nodes

Data in plottr is transformed through a flowchart of nodes. This is inherited from the pyqtgraph library. Each node typically has some input data, does some processing on it, which then results in output data. This is done essentially on-demand, i.e., only when the input has changed, the node performs processing. 

## Data model

Within plotter, we use the DataDict format (see the "Working with data" example notebook). This is basically a dictionary-based data container with some standardized meta-data and some convenience functions for working on the data. Each node within plottr is expected to accept DataDicts as input and to produce DataDicts as output (with the exception of None, which signals that there is no output right now).

# Setting up

## Required libraries

Under the hood we make heavy use of pyqt and pyqtgraph; for plotting we use matplotlib as well. Our main import block looks like this (see below for the role each of the parts play).
Note that the first code line is the `gui qt` magic, which allows us handle a Qt mainloop nicely within the notebook.

In [2]:
%gui qt

import numpy as np
from matplotlib import pyplot as plt

import pyqtgraph as pg
from pyqtgraph.flowchart import Flowchart
from pyqtgraph import Qt
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.flowchart import library as fclib

# qtconsole is sometimes nice as a complement
# %qtconsole

## Setting up logging

To avoid confusion when handling messages and user information from the flowchart and nodes we use a custom log handler. All log messages sent to a logger with name "plottr.\*" will be captured.

Since plottr is GUI tool, it is only fitting to use a GUI output for all logs :)
Here we open a separate dialog which contains text output from plottr.

In [20]:
import logging
from plottr import log as plottrlog
logDialog = plottrlog.setupLogging()
logger = logging.getLogger('plottr.interactive')
plottrlog.LEVEL = logging.DEBUG

# Selecting and plotting data using code

In [6]:
from plottr.data import datadict; reload(datadict)
from plottr.data.datadict import togrid, DataDict, GridDataDict

from plottr.node import node; reload(node)
from plottr.node.node import Node, NodesWidget

from plottr.node import data_selector; reload(data_selector)
from plottr.node.data_selector import DataSelector

from plottr.node import dim_reducer; reload(dim_reducer)
from plottr.node.dim_reducer import DimensionReducer, XYAxesSelector

from plottr.plot import mpl; reload(mpl)
from plottr.plot.mpl import PlotNode, AutoPlot

In [7]:
def testdata_2d(nx=10, ny=10):
    x = np.linspace(0, 10, nx)
    y = np.arange(ny)
    
    xx, yy = np.meshgrid(x, y, indexing='ij')
    dd = np.cos(xx) + (-0.05 + 0.1 * np.random.rand(*yy.shape))
    dd2 = np.sin(xx) + (-0.5 + 1 * np.random.rand(*yy.shape))
    
    d = DataDict(
        x = dict(values=xx.reshape(-1)),
        y = dict(values=yy.reshape(-1)),
        cos_data = dict(values=dd.reshape(-1), axes=['x', 'y']),
        sin_data = dict(values=dd2.reshape(-1), axes=['x', 'y']),
    )
    return d

data_in = testdata_2d(21, 500)

In [8]:
Node.raiseExceptions = True
Node.debug = True
DataSelector.debug = True
DataSelector.useUi = False

nodelib = fclib.NodeLibrary()
nodelib.addNodeType(DataSelector, [('Basic')])
nodelib.addNodeType(DimensionReducer, [('Basic')])
nodelib.addNodeType(XYAxesSelector, [('Basic')])
nodelib.addNodeType(PlotNode, [('Plot')])

fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}
})
fc.library = nodelib

selector = fc.createNode('DataSelector')
dim_reduce = fc.createNode('DimensionReducer')
xysel = fc.createNode('XYAxesSelector')
plot = fc.createNode('Plot')

fc.connectTerminals(fc['dataIn'], selector['dataIn'])
# fc.connectTerminals(selector['dataOut'], dim_reduce['dataIn'])
# fc.connectTerminals(dim_reduce['dataOut'], fc['dataOut'])
fc.connectTerminals(selector['dataOut'], xysel['dataIn'])
fc.connectTerminals(xysel['dataOut'], fc['dataOut'])
fc.connectTerminals(xysel['dataOut'], plot['dataIn'])


plotWidget = AutoPlot()
plot.setPlotWidget(plotWidget)

dialog = QtGui.QDialog()
# nw = NodesWidget()
# nw.addNodeWidget(selector, 'Data Selector')
layout = QtGui.QVBoxLayout(dialog)
layout.addWidget(plotWidget)
dialog.show()

# fc.setInput(dataIn=data_in)

In [9]:
selector.grid = True
# selector.selectedData = [
#     'sin_data', 
#     'cos_data',
# ]

# dim_reduce.targetNames = ['sin_data']
# dim_reduce.reductions = {
#     'y' : (np.mean, ),
# }

In [10]:
# xysel.xyAxes = 'x', 'y'
# xysel.reductions = {
# #     'y' : (dim_reducer.selectAxisElement, [0],),
# #     'y' : (np.mean, ),
# #     'y' : (dim_reducer.sliceAxis, [np.s_[0:2:]], ),
# }

data_out = fc.output()['dataOut']

# fig, ax = plt.subplots(1,1)
# _ = ax.plot(data_out['x']['values'], data_out['cos_data']['values'], 'r.')
# _ = ax.plot(data_out['x']['values'], data_out['sin_data']['values'], 'b.')

In [11]:
fc.setInput(dataIn=data_in)

In [12]:
selector.selectedData = [
    'sin_data', 
#     'cos_data',
]

In [13]:
xysel.xyAxes = 'x', 'y'
# xysel.reductions = {
# #     'y' : (dim_reducer.selectAxisElement, [0],),
# #     'y' : (np.mean, ),
# #     'y' : (dim_reducer.sliceAxis, [np.s_[0:2:]], ),
# }

In [14]:
fc.output()

{'dataOut': {'x': {'values': array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
           5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. ]),
   'unit': '',
   'axes': [],
   'info': {}},
  'y': {'values': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
           13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
           26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
           39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
           52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
           65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
           78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
           91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
          104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
          117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
          130, 